In [1]:
# notebooks/05_RoBERTa.ipynb
import os
import logging
import sys

# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils.file_utils import read_excel_file, save_data_to_excel
from utils.advanced_ner_utils import perform_ner_with_roberta

from config import XLSX_DIRECTORY
from utils.ner_utils import cluster_entities, process_sentences_and_extract_entities
from transformers import AutoTokenizer, AutoModelForTokenClassification


# Carga de modelos y tokenizers
roberta_model_name = "PlanTL-GOB-ES/roberta-large-bne-capitel-ner"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForTokenClassification.from_pretrained(roberta_model_name)


def main():
    # Cargar los datos de las oraciones desde el archivo Excel
    logging.info("Cargando datos de oraciones desde Excel...")
    sentence_data = read_excel_file(os.path.join(XLSX_DIRECTORY, "sentence.xlsx"))

    # Realizar NER con RoBERTa
    logging.info("Realizando NER con RoBERTa...")
    roberta_entities_df = process_sentences_and_extract_entities(
        sentence_data,
        roberta_tokenizer,
        roberta_model,
        perform_ner_with_roberta,
        "roberta",
    )

    # Convertir DataFrame de entidades a lista de diccionarios para el clustering
    roberta_entities_df = roberta_entities_df.to_dict("records")

    # Opcional: Agrupar entidades similares con K-means
    logging.info("Agrupando entidades similares...")
    roberta_entities_df = cluster_entities(roberta_entities_df, num_clusters=10)

    # Guardar los datos de entidades RoBERTa en un nuevo archivo Excel
    logging.info("Guardando datos de entidades RoBERTa en Excel...")
    save_data_to_excel(roberta_entities_df, XLSX_DIRECTORY, "roberta_entity.xlsx")


if __name__ == "__main__":
    main()

Procesando: 0/2208 oraciones (0.00%)
Procesando: 220/2208 oraciones (9.96%)
Procesando: 440/2208 oraciones (19.93%)
Procesando: 660/2208 oraciones (29.89%)
Procesando: 880/2208 oraciones (39.86%)
Procesando: 1100/2208 oraciones (49.82%)
Procesando: 1320/2208 oraciones (59.78%)
Procesando: 1540/2208 oraciones (69.75%)
Procesando: 1760/2208 oraciones (79.71%)

omitiendo Entidad 'Decreto de Urgencia N�� 017' no encontrada en el texto.
Contexto: efectos del COVID - 19 [ENTIDAD] mediante el Decreto de Urgencia
Oración: No obstante, lo anterior, mediante el Decreto de Urgencia Nº 017-2022 se han destinado 96.8 millones de soles para atender las demandas de las ollas comunes y enfrentar el incremento de los precios producto del contexto internacional y de los efectos del COVID-19.
Procesando: 1980/2208 oraciones (89.67%)
Procesando: 2200/2208 oraciones (99.64%)
Procesamiento completado.


d:\TFM_Project\PLN_Project\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
